## Example: using Linear Programming to solve for 3 latent factors `w`

Please note that the last part of this example, "Permutation invariance in linear programming vs. MultiCCA in R and `pmd`" requires that `inpt3` be exactly as printed.

In [1]:
import pandas as pd
import numpy as np

from sparsecca import lp_pmd

In [2]:
# example inputs
inpt1 = pd.read_csv("../tests/data/multicca1.csv", sep=",", index_col=0).values
inpt2 = pd.read_csv("../tests/data/multicca2.csv", sep=",", index_col=0).values[:, -5:]
inpt3 = np.random.normal(size=inpt2.shape)

penalties = [1.5, 1.5, 1.5]

In [3]:
assert inpt1.shape[0] == inpt2.shape[0] == inpt3.shape[0]
inpt1.shape

(25, 5)

In [4]:
(inpt1.T @ inpt2)

array([[-90.17027552,  11.82731312,  -7.86532209,  17.62955376,
        -17.56197152],
       [ 51.8255256 ,   6.41554333, -14.01087688, -46.99815802,
         16.29765355],
       [ 74.04185358,  29.92014574, -26.72555091, -42.34996728,
         26.97207374],
       [-47.72640718, -18.39841178, -21.55303292,  37.29813179,
        -27.72045331],
       [ -6.14657028,  18.29437855,   4.43004474,  -2.18546732,
         12.80321423]])

In [5]:
weights, svd_init = lp_pmd(
    datasets=[inpt1, inpt2], # match feature dimension for now
    penalties=penalties[:2],
    K=3,
    standardize=True,
    mimic_R=True
)

In [6]:
# pearson correlation of weights[0] and weights[1]
np.corrcoef(weights[0].flatten(), weights[1].flatten())


array([[ 1.        , -0.83859139],
       [-0.83859139,  1.        ]])

## Correlation between pmd results and lp results

In [7]:
from scipy.stats import spearmanr
from sparsecca import multicca_pmd

### high correlation between pmd and lp when `n=2`

Let `n` equal the number of datasets.

In [8]:
datasets = [inpt1, inpt2]
ws_lp, _ = lp_pmd(datasets, penalties[:2])
ws_r, _ = multicca_pmd(datasets, penalties[:2])

In [9]:
print(spearmanr(ws_lp[0], ws_r[0]))
print(spearmanr(ws_lp[1], ws_r[1]))

SignificanceResult(statistic=-0.9746794344808964, pvalue=0.004818230468198537)
SignificanceResult(statistic=0.09999999999999999, pvalue=0.8728885715695383)


Note that this is the same as taking the first latent factor when K=3, as per the implementation in Witten 2009.

In [10]:
print(spearmanr(weights[0].T[0], ws_r[0]))
print(spearmanr(weights[1].T[0], ws_r[1]))

SignificanceResult(statistic=-0.9746794344808964, pvalue=0.004818230468198537)
SignificanceResult(statistic=0.09999999999999999, pvalue=0.8728885715695383)


### arbitrarily lower correlation when `n=3`

Compare when `n=3`. Note that this is subject to the same warning as in the bottom of the notebook.

In [11]:
datasets = [inpt1, inpt2, inpt3]
ws_lp, _ = lp_pmd(datasets, penalties)
ws_r, _ = multicca_pmd(datasets, penalties)

In [12]:
print(spearmanr(ws_lp[0], ws_r[0]))
print(spearmanr(ws_lp[1], ws_r[1]))
print(spearmanr(ws_lp[2], ws_r[2]))

SignificanceResult(statistic=-0.8207826816681233, pvalue=0.08858700531354381)
SignificanceResult(statistic=-0.09999999999999999, pvalue=0.8728885715695383)
SignificanceResult(statistic=-0.8207826816681233, pvalue=0.08858700531354381)


#### Optimization of the objective function

Size of the objective function, which was maximized:

In [13]:
print('1st pair:', ws_lp[0].T @ inpt1.T @ inpt2 @ ws_lp[1])
print('2nd pair:', ws_lp[1].T @ inpt2.T @ inpt3 @ ws_lp[2])
print('3rd pair:', ws_lp[2].T @ inpt3.T @ inpt1 @ ws_lp[0])
print('sum', ws_lp[0].T @ inpt1.T @ inpt2 @ ws_lp[1] + ws_lp[1].T @ inpt2.T @ inpt3 @ ws_lp[2] + ws_lp[2].T @ inpt3.T @ inpt1 @ ws_lp[0])

1st pair: [[2.36359908e-06]]
2nd pair: [[5.40590458e-07]]
3rd pair: [[51.33244746]]
sum [[51.33245036]]


In [14]:
print('1st pair:', ws_r[0].T @ inpt1.T @ inpt2 @ ws_r[1])
print('2nd pair:', ws_r[1].T @ inpt2.T @ inpt3 @ ws_r[2])
print('3rd pair:', ws_r[2].T @ inpt3.T @ inpt1 @ ws_r[0])
print('sum', ws_r[0].T @ inpt1.T @ inpt2 @ ws_r[1] + ws_r[1].T @ inpt2.T @ inpt3 @ ws_r[2] + ws_r[2].T @ inpt3.T @ inpt1 @ ws_r[0])

1st pair: [[99.11315683]]
2nd pair: [[41.60492912]]
3rd pair: [[31.5520592]]
sum [[172.27014515]]


L2 norm of the latent factors, constrained.

In [15]:
np.linalg.norm(ws_lp)

1.4142135829891374

In [16]:
np.linalg.norm(ws_r)

1.7320508075688772

Neither linear programming nor manual convergence consistently produces a better optimization of the objective function in this small test case.

## Permutation invariance in linear programming vs. MultiCCA in R and `pmd`

In [17]:
def test_weights(weights_a, weights_b, perm_b: list[int], dec=5):
    """Tests whether `weights_a` and `weights_b` are the same given the permutation order of b.

    Parameters:
        weights_a: output of lp_pmd 
        weights_b: output of lp_pmd permuted Xn
                   -> weights are of type np.ndarray in shape (N, f, K)
                    - N: len(Xn) datasets
                    - f: amount of features
                    - K: amount of MCPs
        perm_b:    order of the datasets used to generate a, in b
        dec:       decimals to which weights should be rounded to account for numerical tolerance

    Returns:
        boolean: True if rounded weights are the same, else False
    """
    
    weights_a_rounded = np.array(weights_a).round(decimals=dec)
    weights_b_rounded = np.array(weights_b).round(decimals=dec)
    
    weights_b_ordered = []
    for o in perm_b:
        weights_b_ordered.append(weights_b_rounded[o])
        
    return all(x==True for x in (weights_a_rounded==weights_b_ordered).flatten())

In [18]:
datasets = [inpt1, inpt2, inpt3]
# original dataset with perm 1, 2, 0
datasets_perm = [inpt3, inpt1, inpt2]

### linear programming

In [19]:
threshold = 1e-10

ws_lp, _ = lp_pmd(datasets, [0.4, 0.0, 0.4])
ws_lp = np.array(ws_lp)
ws_lp[np.isclose(ws_lp, 0, rtol=threshold)] = 0

ws_lp_perm, _ = lp_pmd(datasets_perm, [0.4, 0.0, 0.4])
ws_lp_perm = np.array(ws_lp_perm)
ws_lp_perm[np.isclose(ws_lp, 0, rtol=threshold)] = 0

In [20]:
ws_lp

array([[[-6.09416697e-01],
        [-7.92850123e-01],
        [ 0.00000000e+00],
        [ 0.00000000e+00],
        [ 0.00000000e+00]],

       [[ 0.00000000e+00],
        [ 0.00000000e+00],
        [ 0.00000000e+00],
        [ 8.88854528e-08],
        [ 0.00000000e+00]],

       [[ 8.69830201e-01],
        [ 0.00000000e+00],
        [ 0.00000000e+00],
        [-4.93351244e-01],
        [ 0.00000000e+00]]])

In [21]:
ws_lp_perm

array([[[-5.73418393e-18],
        [-1.01421456e-17],
        [ 0.00000000e+00],
        [ 0.00000000e+00],
        [ 0.00000000e+00]],

       [[ 0.00000000e+00],
        [ 0.00000000e+00],
        [ 0.00000000e+00],
        [ 3.46602125e-11],
        [ 0.00000000e+00]],

       [[ 8.62126560e-01],
        [ 0.00000000e+00],
        [ 0.00000000e+00],
        [-5.06693026e-01],
        [ 0.00000000e+00]]])

In [22]:
test_weights(ws_lp, ws_lp_perm, [1,2,0], dec=5)

False

As we can see, the weights are the same with merely the order permuted as is appropriate.

### R (in python)

In [23]:
ws_r, _ = multicca_pmd(datasets, penalties)
ws_r_perm, _ = multicca_pmd(datasets_perm, penalties)

In [24]:
ws_r

[array([[ 0.        ],
        [ 0.90824506],
        [ 0.29728456],
        [-0.29447039],
        [ 0.        ]]),
 array([[ 0.56055407],
        [-0.07202121],
        [ 0.04359745],
        [-0.82382725],
        [ 0.        ]]),
 array([[-0.7682389 ],
        [-0.09935492],
        [-0.63240619],
        [ 0.        ],
        [-0.        ]])]

In [25]:
ws_r_perm

[array([[-0.76824118],
        [-0.09935549],
        [-0.63240334],
        [ 0.        ],
        [-0.        ]]),
 array([[ 0.        ],
        [ 0.90824509],
        [ 0.29727909],
        [-0.29447581],
        [ 0.        ]]),
 array([[ 0.5605532 ],
        [-0.0720213 ],
        [ 0.04359766],
        [-0.82382782],
        [ 0.        ]])]

Test with decimal tolerance 5.

In [26]:
for i in range(len(ws_r)):
    print("IO penalties:", 1 - np.count_nonzero(ws_r[i]) / ws_r[i].size)
    print("LP penalties:", 1 - np.count_nonzero(ws_lp[i]) / ws_lp[i].size)


IO sparsity: 0.4
LP sparsity: 0.6
IO sparsity: 0.19999999999999996
LP sparsity: 0.8
IO sparsity: 0.4
LP sparsity: 0.6


In [27]:
test_weights(np.array(ws_r), np.array(ws_r_perm), [1,2,0])

False

Test with decimal tolerance 1.

In [28]:
test_weights(np.array(ws_r), np.array(ws_r_perm), [1,2,0], dec=1)

True

Test the equivalent negative solution.

In [29]:
test_weights(np.array(ws_r), -np.array(ws_r_perm), [1,2,0], dec=1)

False

**Note**: linear programming will always solve for the same objective function regardless of the order of the inputs, but the custom PMD implementation in `multicca_pmd` will not. Depending on the dataset, the solution may converge similarly regardless of order (with only a difference in sign), or the solution may converge to a completely different local minima given a different order.